In [5]:
import pandas as pd

# === 📂 Chemins des fichiers ===
OUTPUT_COMBINED = "output/agences_combined_lokker_ready.csv"
INPUT_CM = "agences_cm_geocoded/credit_mutuel_agences_full_progress.csv"

# === 🔁 Chargement des deux datasets ===
df_combined = pd.read_csv(OUTPUT_COMBINED)
df_cm = pd.read_csv(INPUT_CM)

# === ➕ Ajout de la colonne source manquante pour le Crédit Mutuel ===
df_cm['source'] = "Crédit Mutuel"

# === 🧹 Harmonisation (si besoin : nettoyage d'espaces ou doublons) ===
df_cm = df_cm[['nom', 'adresse', 'code_postal', 'latitude', 'longitude', 'region_source', 'source']]
df_combined = df_combined[['nom', 'adresse', 'code_postal', 'latitude', 'longitude', 'region_source', 'source']]

# === 🔗 Fusion
df_merged = pd.concat([df_combined, df_cm], ignore_index=True)

# === ❌ Suppression des lignes sans coordonnées
df_merged = df_merged.dropna(subset=['latitude', 'longitude'])

# === 💾 Sauvegarde
df_merged.to_csv(OUTPUT_COMBINED, index=False)
print("✅ Fusion avec Crédit Mutuel terminée et enregistrée dans :", OUTPUT_COMBINED)


✅ Fusion avec Crédit Mutuel terminée et enregistrée dans : output/agences_combined_lokker_ready.csv


In [ ]:
import pandas as pd
import pyproj

# === 📂 Fichiers ===
INPUT_CE = "agences_ce/agences-caisse-depargne0.csv"
OUTPUT_COMBINED = "output/agences_combined_lokker_ready.csv"

# === 1. Charger les données Caisse d'Épargne ===
df_ce = pd.read_csv(INPUT_CE, sep=';', dtype=str)

# === 2. Conversion Lambert II vers Latitude/Longitude ===
proj_lambert2 = pyproj.Proj(init="epsg:27572")  # Lambert II étendu
proj_wgs84 = pyproj.Proj(init="epsg:4326")

def convert_lambert_to_latlon(x, y):
    try:
        lon, lat = pyproj.transform(proj_lambert2, proj_wgs84, float(x), float(y))
        return lat, lon
    except:
        return None, None

df_ce[['latitude', 'longitude']] = df_ce[['X_lambert_2', 'Y_lambert_2']].apply(
    lambda row: pd.Series(convert_lambert_to_latlon(row['X_lambert_2'], row['Y_lambert_2'])),
    axis=1
)

# === 3. Standardiser les colonnes pour merge ===
df_ce['nom'] = df_ce['nom_agence']
df_ce['adresse'] = df_ce['adresse1'].fillna('') + ' ' + df_ce['adresse2'].fillna('')
df_ce['code_postal'] = df_ce['code_postal']
df_ce['region_source'] = df_ce['libelle_caisse_de_rattachement']
df_ce['source'] = "Caisse d'Épargne"

# === 4. Garder uniquement les colonnes standardisées ===
df_ce_clean = df_ce[['nom', 'adresse', 'code_postal', 'latitude', 'longitude', 'region_source', 'source']]

# === 5. Charger le fichier existant et concaténer ===
df_combined = pd.read_csv(OUTPUT_COMBINED)
df_merged = pd.concat([df_combined, df_ce_clean], ignore_index=True)

# === 6. Nettoyer les NaN si nécessaire ===
df_merged = df_merged.dropna(subset=['latitude', 'longitude'])

# === 7. Sauvegarder ===
df_merged.to_csv(OUTPUT_COMBINED, index=False)
print("✅ Fusion terminée et enregistrée dans :", OUTPUT_COMBINED)


c:\Users\iroum\AppData\Local\Programs\Python\Python313\Lib\site-packages\pyproj\crs\crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\iroum\AppData\Local\Programs\Python\Python313\Lib\site-packages\pyproj\crs\crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


KeyError: "None of [Index(['X_lambert_2', 'Y_lambert_2'], dtype='object')] are in the [columns]"

In [8]:
import pandas as pd

# === 📂 Fichiers
INPUT_BP = "agences_bp/agences-banque-populaire.csv"
OUTPUT_COMBINED = "output/agences_combined_lokker_ready.csv"

# === 🔁 Lecture des fichiers
df_bp = pd.read_csv(INPUT_BP, sep=';', dtype=str)
df_combined = pd.read_csv(OUTPUT_COMBINED)

# === 🧼 Nettoyage & transformation coordonnées
# Séparer "lat,lon" en deux colonnes
df_bp[['latitude', 'longitude']] = df_bp['coordonnees_gps'].str.split(',', expand=True)

# Nettoyer les champs
df_bp['nom'] = df_bp['Nom_Guichet']
df_bp['adresse'] = df_bp['Rue'].fillna('') + ' ' + df_bp['Complement_adresse'].fillna('') + ' ' + df_bp['Ville'].fillna('')
df_bp['code_postal'] = df_bp['Code_Postal']
df_bp['source'] = df_bp['Libellé_Marque']  # ex: Banque Populaire
df_bp['region_source'] = ""  # facultatif ou à calculer si tu veux

# Ne garder que les colonnes nécessaires
df_bp_clean = df_bp[['nom', 'adresse', 'code_postal', 'latitude', 'longitude', 'region_source', 'source']]

# Convertir les coordonnées en float
df_bp_clean['latitude'] = pd.to_numeric(df_bp_clean['latitude'], errors='coerce')
df_bp_clean['longitude'] = pd.to_numeric(df_bp_clean['longitude'], errors='coerce')

# Supprimer les lignes sans coordonnées valides
df_bp_clean = df_bp_clean.dropna(subset=['latitude', 'longitude'])

# === 🔗 Fusion
df_final = pd.concat([df_combined, df_bp_clean], ignore_index=True)

# === 💾 Sauvegarde
df_final.to_csv(OUTPUT_COMBINED, index=False)
print("✅ Banque Populaire ajoutée à :", OUTPUT_COMBINED)


✅ Banque Populaire ajoutée à : output/agences_combined_lokker_ready.csv


C:\Users\iroum\AppData\Local\Temp\ipykernel_2476\3928741125.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bp_clean['latitude'] = pd.to_numeric(df_bp_clean['latitude'], errors='coerce')
C:\Users\iroum\AppData\Local\Temp\ipykernel_2476\3928741125.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bp_clean['longitude'] = pd.to_numeric(df_bp_clean['longitude'], errors='coerce')


In [39]:
#updt by the lat lon found from geocoding from the postal code
#change region souce to "Banque Populaire" pour all rows
import pandas as pd
df = pd.read_csv("Final_clean/agences-ce_merged_pb.csv")
df['region_source'] = "Caisse d'Épargne"
df.to_csv("Final_clean/agences-ce_merged_pb.csv", index=False)

In [ ]:
df.iloc[115, df.columns.get_indexer(['latitude', 'longitude'])] = [49.21667, 1.16667]
df.iloc[333, df.columns.get_indexer(['latitude', 'longitude'])] = [45.4411934, 4.3859445]
df.iloc[358, df.columns.get_indexer(['latitude', 'longitude'])] = [46.84314727783203, -0.4944003224372864]
df.iloc[607, df.columns.get_indexer(['latitude', 'longitude'])] = [47.2711217, -1.623206]
df.to_csv("agences-ce_merged_pb_geocoded/agences-ce_merged_pb.csv", index=False)

In [37]:
#open agences-banque-populaire_normalized.csv and get the lat missing values from geocoding
# --- IGNORE ---
import pandas as pd
df = pd.read_csv("agences-ce_merged_pb_geocoded/agences-ce_merged_pb.csv")
df[df['latitude'].isna()]


,nom,adresse,code_postal,latitude,longitude,region_source


In [7]:

import pandas as pd

# Dictionnaire code postal → (latitude, longitude)
cp_coords = {
    "69337": (45.7597, 4.8422),
    "91042": (48.6239, 2.4294),
    "75013": (48.8296, 2.3570),
    "25051": (47.2488, 6.0182),
    "34078": (43.6111, 3.8767),
    "38490": (45.5236, 5.5850),
    "05100": (44.8992, 6.6427),
    "38217": (45.5161, 4.8744),
    "69211": (45.7600, 4.8500),
    "42007": (45.4397, 4.3872),
    "57021": (49.1193, 6.1757),
    "78183": (48.7742, 2.0455),
    "63172": (45.7708, 3.0872),
    "43102": (45.2961, 3.3836),
    "43010": (45.0424, 3.8829),
    "42161": (45.5286, 4.2608),
    "75204": (48.8566, 2.3522),
    "21000": (47.3220, 5.0415),
    "75005": (48.8462, 2.3526),
    "60600": (49.3831, 2.4163),
    "15017": (44.9260, 2.4409),
    "42308": (46.0339, 4.0681),
    "42005": (45.4397, 4.3872),
}

df = pd.read_csv("agences_banque_populaire_geocoded/agences-banque-populaire_normalized.csv")

# Remplir les lat/lon manquantes selon le code postal
def fill_lat_lon(row):
    if pd.isna(row['latitude']) and str(row['code_postal']) in cp_coords:
        lat, lon = cp_coords[str(row['code_postal'])]
        row['latitude'] = lat
        row['longitude'] = lon
    return row

df = df.apply(fill_lat_lon, axis=1)

df.to_csv("agences_banque_populaire_geocoded/agences-banque-populaire_normalized.csv", index=False)
print("✅ Coordonnées manquantes complétées selon le code postal.")

✅ Coordonnées manquantes complétées selon le code postal.


In [40]:
#change Final_clean/agences_bnp_merged_pb_fixed.csv csv to execl
import pandas as pd
import csv
df = pd.read_csv("Final_clean/agences-ce_merged_pb.csv",
                 sep=",", engine="python", encoding="utf-8-sig",
                 quotechar='"', quoting=csv.QUOTE_MINIMAL, escapechar="\\")

df.to_excel("Final_clean/agences_ce_merged_pb.xlsx", index=False)
